In [2]:
# Below is the modified Python code to handle 5-minute intervals, 
# incorporating solar fluctuations and aggregating results over 1 hour for billing.
# 23 April 25
import numpy as np

# System Parameters
# Loads (kWh, per 5-minute interval, assumed constant)
L = np.array([100/12, 50/12, 150/12, 160/12])  # [8.333, 4.167, 12.5, 13.333]

# Average Solar Production (kWh, per 5-minute interval)
S_avg = np.array([200/12, 60/12, 20/12, 20/12])  # [16.667, 5, 1.667, 1.667]

# Number of buildings and intervals
n_buildings = 4
n_intervals = 12  # 12 x 5-minute intervals = 1 hour

# Simulate solar fluctuations (±30% around average)
np.random.seed(42)  # For reproducibility
fluctuation = 0.3  # ±30%
S_intervals = np.zeros((n_intervals, n_buildings))
for t in range(n_intervals):
    for i in range(n_buildings):
        S_intervals[t, i] = S_avg[i] * (1 + np.random.uniform(-fluctuation, fluctuation))
        S_intervals[t, i] = round(S_intervals[t, i], 3)

# Initialize arrays to store results
E_total = np.zeros((n_buildings, n_buildings))  # Total transfers over 1 hour
M_total = np.zeros(n_buildings)  # Total MDB readings
M5_total = 0  # Total grid import

# Process each 5-minute interval
for t in range(n_intervals):
    print(f"\nInterval {t+1} (5 minutes):")
    S = S_intervals[t, :]
    print(f"Solar Production (kWh): {S}")

    # Step 1: Calculate Surplus and Deficit
    surplus_deficit = S - L
    surplus = np.zeros(n_buildings)
    deficit = np.zeros(n_buildings)
    for i in range(n_buildings):
        if surplus_deficit[i] > 0:
            surplus[i] = surplus_deficit[i]
        else:
            deficit[i] = -surplus_deficit[i]
    
    total_surplus = np.sum(surplus)
    total_deficit = np.sum(deficit)
    print(f"Surplus (kWh): {surplus}")
    print(f"Deficit (kWh): {deficit}")
    print(f"Total Surplus: {total_surplus:.3f} kWh, Total Deficit: {total_deficit:.3f} kWh")

    # Step 2: Pre-transfer MDB Readings
    M_pre = np.zeros(n_buildings)
    for i in range(n_buildings):
        M_pre[i] = S[i] - L[i]
    print(f"Pre-transfer MDB (kWh): {M_pre}")

    # Step 3: Allocate Transfers
    E = np.zeros((n_buildings, n_buildings))
    if total_deficit > 0:
        deficit_shares = np.zeros(n_buildings)
        for i in range(n_buildings):
            if deficit[i] > 0:
                deficit_shares[i] = deficit[i] / total_deficit
        
        for i in range(n_buildings):
            if M_pre[i] < 0:
                surplus_i = -M_pre[i]
                for j in range(n_buildings):
                    if deficit[j] > 0:
                        E[i, j] = deficit_shares[j] * surplus_i
                        E[i, j] = round(E[i, j], 3)
    
    print(f"Transfers (kWh):")
    for i in range(n_buildings):
        for j in range(n_buildings):
            if E[i, j] > 0:
                print(f"E_{i+1}->{j+1}: {E[i, j]:.3f}")

    # Step 4: Calculate MDB Meter Readings
    M = np.zeros(n_buildings)
    for i in range(n_buildings):
        received = np.sum(E[:, i])
        exported = np.sum(E[i, :])
        M[i] = L[i] - S[i] - received + exported
        M[i] = round(M[i], 3)
    
    M5 = np.sum(M)
    print(f"MDB Readings (kWh): {M}")
    print(f"Grid Import (M_5): {M5:.3f} kWh")

    # Step 5: Verify Billing with Final M_j
    total_import = np.sum(M[M > 0])
    if total_import > 0:
        import_shares = np.zeros(n_buildings)
        for j in range(n_buildings):
            if M[j] > 0:
                import_shares[j] = M[j] / total_import
        
        E_final = np.zeros((n_buildings, n_buildings))
        for i in range(n_buildings):
            if M_pre[i] < 0:
                surplus_i = -M_pre[i]
                for j in range(n_buildings):
                    if M[j] > 0:
                        E_final[i, j] = import_shares[j] * surplus_i
                        E_final[i, j] = round(E_final[i, j], 3)
    else:
        E_final = E
    
    # Accumulate results
    E_total += E_final
    M_total += M
    M5_total += M5

# Step 6: Aggregate for Hourly Billing
print("\nHourly Billing (Aggregated over 12 Intervals):")
print(f"Total Loads (kWh): {L * n_intervals}")
print(f"Total Solar Production (kWh): {np.sum(S_intervals, axis=0)}")
print(f"\nTotal Transfers (kWh):")
for i in range(n_buildings):
    for j in range(n_buildings):
        if E_total[i, j] > 0:
            print(f"E_{i+1}->{j+1}: {E_total[i, j]:.3f}")
print(f"\nTotal MDB Readings (kWh): {M_total}")
print(f"Total Grid Import (M_5): {M5_total:.3f} kWh")

print("\nBilling Outcome:")
print("Inter-building Billing:")
for j in range(n_buildings):
    if np.sum(E_total[:, j]) > 0:
        print(f"Building {j+1}:")
        for i in range(n_buildings):
            if E_total[i, j] > 0:
                print(f"  {E_total[i, j]:.3f} kWh from Building {i+1}")
for i in range(n_buildings):
    total_exported = np.sum(E_total[i, :])
    if total_exported > 0:
        print(f"Building {i+1}: Credited for {total_exported:.3f} kWh")
print("\nGrid Billing:")
for i in range(n_buildings):
    print(f"Building {i+1}: {M_total[i]:.3f} kWh")
print(f"Total Grid Import: {M5_total:.3f} kWh")


Interval 1 (5 minutes):
Solar Production (kWh): [15.412  6.352  1.899  1.765]
Surplus (kWh): [7.07866667 2.18533333 0.         0.        ]
Deficit (kWh): [ 0.          0.         10.601      11.56833333]
Total Surplus: 9.264 kWh, Total Deficit: 22.169 kWh
Pre-transfer MDB (kWh): [  7.07866667   2.18533333 -10.601      -11.56833333]
Transfers (kWh):
E_3->3: 5.069
E_3->4: 5.532
E_4->3: 5.532
E_4->4: 6.037
MDB Readings (kWh): [-7.079 -2.185 10.601 11.568]
Grid Import (M_5): 12.905 kWh

Interval 2 (5 minutes):
Solar Production (kWh): [13.227  3.968  1.225  2.033]
Surplus (kWh): [4.89366667 0.         0.         0.        ]
Deficit (kWh): [ 0.          0.19866667 11.275      11.30033333]
Total Surplus: 4.894 kWh, Total Deficit: 22.774 kWh
Pre-transfer MDB (kWh): [  4.89366667  -0.19866667 -11.275      -11.30033333]
Transfers (kWh):
E_2->2: 0.002
E_2->3: 0.098
E_2->4: 0.099
E_3->2: 0.098
E_3->3: 5.582
E_3->4: 5.595
E_4->2: 0.099
E_4->3: 5.595
E_4->4: 5.607
MDB Readings (kWh): [-4.894  0.199